**Precondition:** Download the *"186 Watershed boundary"* files from [NSDI Bhutan](https://nsdi.systems.gov.bt/data/Boundaries) and save them in `../../data/186_Watershed_boundary/`.

In [ ]:
from pathlib import Path
import pandas as pd
import os
os.environ["GEOPANDAS_IO_ENGINE"] = "pyogrio"   # або просто передати engine=
import geopandas as gpd
import rasterio   #This is a library for working with raster geospatial data (GeoTIFF)
from rasterstats import zonal_stats #It allows to calculate statistics on raster values (for example, a DEM) within vector polygons


dem_path = Path("../../data/HydroSHEDS/as_dem_Bhutan_and_buffer.tif") #DEM data from HydroSHEDS
polygon_coordinates_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.shp") #186 Watershed boundary.shp → geometry (polygon coordinates)
indexes_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.shx") #indexes for quick access
attribute_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.dbf") #attribute table (ID, other fields)
coordinate_system_path = Path("../../data/186_Watershed_boundary/186 Watershed boundary.prj") #To align Coordinate Reference System with DEM

if not (dem_path.exists() == polygon_coordinates_path.exists() == indexes_path.exists() == attribute_path.exists() == coordinate_system_path.exists() ==True):
    raise FileNotFoundError("One or more required files are missing. Please check the paths.")

# -----Check if DEM and 186_Watershed have the same CRM -----
ws = gpd.read_file(polygon_coordinates_path, engine="pyogrio")
             
with rasterio.open(dem_path) as r:
    dem_crs = r.crs
    dem_nodata = r.nodata

if not ws.crs == dem_crs:
    print ("CRS NOT match!")
    print("WS CRS:", ws.crs)
    print("DEM CRS:", dem_crs)

CRS NOT match!
WS CRS: EPSG:3857
DEM CRS: EPSG:4326
